In [418]:
library(foreign)
library(tidyr)
library(dplyr)
library(lcmm)

Loading required package: survival

Loading required package: parallel



In [235]:
cudos = read.spss('../Data/DailyMeasures/CUDOS_1.sav', reencode = NA, to.data.frame = T)
cudos = as.data.frame(lapply(cudos, as.numeric))
cudos = cudos[ ,-grep('depF', colnames(cudos))]
log = read.spss('../Data/DemosDx/Tx Day Consent Log_1.sav', to.data.frame = T)

Warning message in read.spss("../Data/DailyMeasures/CUDOS_1.sav", reencode = NA, :
“Undeclared level(s) 11 added in variable: dep6_14_1”
Warning message in read.spss("../Data/DailyMeasures/CUDOS_1.sav", reencode = NA, :
“Undeclared level(s) 9 added in variable: dep8_1_1”
Warning message in read.spss("../Data/DailyMeasures/CUDOS_1.sav", reencode = NA, :
“Undeclared level(s) 5 added in variable: dep16_1_1”
Warning message in read.spss("../Data/DailyMeasures/CUDOS_1.sav", reencode = NA, :
“Undeclared level(s) 11 added in variable: depF_6_1”
Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav", to.data.frame = T):
“Undeclared level(s) 0, 11, 12 added in variable: DC_status_1”
Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav", to.data.frame = T):
“Duplicated levels in factor DC_status_1: Other”
Warning message in read.spss("../Data/DemosDx/Tx Day Consent Log_1.sav", to.data.frame = T):
“Undeclared level(s) 2 added in variable: SCID_1”
Warning message

In [254]:
cudosLog = merge(cudos, log[,c('ID1','SCID_1','Days_complete_1')], by = 'ID1', all.x = T)
cudosLog

ID1,dep0_1_1,dep0_2_1,dep0_3_1,dep0_4_1,dep0_5_1,dep0_6_1,dep0_7_1,dep0_8_1,dep0_9_1,⋯,dep59_11_1,dep59_12_1,dep59_13_1,dep59_14_1,dep59_15_1,dep59_16_1,dep59_17_1,dep59_18_1,SCID_1,Days_complete_1
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,no,7
2,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,no,7
3,5,4,5,1,5,1,3,3,4,⋯,NA,NA,NA,NA,NA,NA,NA,NA,yes,6
4,5,5,5,1,4,1,5,2,5,⋯,NA,NA,NA,NA,NA,NA,NA,NA,yes,26
5,5,5,1,5,5,1,1,5,5,⋯,NA,NA,NA,NA,NA,NA,NA,NA,yes,8
6,3,3,4,1,1,2,3,3,5,⋯,NA,NA,NA,NA,NA,NA,NA,NA,yes,16
7,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,yes,3
8,5,4,5,1,5,1,3,4,4,⋯,NA,NA,NA,NA,NA,NA,NA,NA,yes,12
9,1,2,4,1,4,1,5,3,3,⋯,NA,NA,NA,NA,NA,NA,NA,NA,yes,7


In [255]:
stem(cudosLog$Days_complete_1)


  The decimal point is 1 digit(s) to the right of the |

  0 | 00000000000000000000000000000000000000000000000000000000000000000000+1452
  0 | 55555555555555555555555555555555555555555555555555555555555555555555+2434
  1 | 00000000000000000000000000000000000000000000000000000000000000000000+1216
  1 | 55555555555555555555555555555555555555555555555555555555555555555555+383
  2 | 00000000000000000000000000000000000000001111111111111111111111111111+56
  2 | 55555555555566666666777777777888899999
  3 | 00000112233444
  3 | 5555778
  4 | 3
  4 | 6
  5 | 
  5 | 89
  6 | 
  6 | 
  7 | 
  7 | 8



In [258]:
cudosLog = cudosLog[which(cudosLog$Days_complete_1 <= 59),] 
meanCompDay = mean(cudosLog$Days_complete_1, na.rm = T) 
print(paste('Completion Day Mean:', round(meanCompDay, 2)))
print(paste('Completion Day SD:', round(sd(cudosLog$Days_complete_1, na.rm = T), 2)))
print(paste('Completion Day Range:', min(cudosLog$Days_complete_1, na.rm = T), 
            max(cudosLog$Days_complete_1, na.rm = T)))



[1] "Completion Day Mean: 8.11"
[1] "Completion Day SD: 5.31"
[1] "Completion Day Range: 0 59"


In [261]:
cudosLog = as.data.frame(lapply(cudosLog, as.numeric))

In [309]:
cudosClean = cudosLog[-which(rowSums(is.na(cudosLog[,grep('dep0', colnames(cudosLog))])) < 18 & cudosLog$SCID_1 == 1),]
cudosClean = cudosClean[-which(rowSums(is.na(cudosClean[,grep('dep0', colnames(cudosClean))])) == 18 & cudosClean$SCID_1 != 1),]

In [326]:
cudosClean

,dep0_1_1,dep1_1_1,dep2_1_1,dep3_1_1,dep4_1_1,dep5_1_1,dep6_1_1,dep7_1_1,dep8_1_1,dep9_1_1,⋯,dep50_1_1,dep51_1_1,dep52_1_1,dep53_1_1,dep54_1_1,dep55_1_1,dep56_1_1,dep57_1_1,dep58_1_1,dep59_1_1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,NA,1,1,1,1,1,1,1,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,NA,5,4,NA,3,2,NA,1,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,5,NA,4,4,3,3,3,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,5,NA,5,5,5,5,5,5,5,5,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,5,NA,4,5,5,5,5,5,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,3,3,4,5,4,5,4,4,5,4,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,5,NA,5,4,5,4,5,5,4,5,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,1,NA,2,2,2,3,3,4,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
13,NA,3,2,3,2,2,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [398]:
select = cudosClean[,grep('ID|_1_1$', colnames(cudosLog))]
cudosLong = select %>% gather('day', 'dep1', -1,-62)

for (i in 2:18){
    select = cudosClean[,grep(paste0('_',i,'_'), colnames(cudosLog))]
    selectLong = select %>% gather('day', 'dep')
    cudosLong = cbind(cudosLong, selectLong$dep)
}


In [399]:
cudosLong$day = rep(0:59, each = 5185)
colnames(cudosLong) = c('ID','SCID','day', paste0('dep', c(1:18)))
cudosLong$day[cudos$SCID==1] 
cudosLong$day[cudosLong$SCID==1] = cudosLong$day[cudosLong$SCID==1] -1
cudosLong = cudosLong[-which(cudosLong$day == -1), ]


integer(0)

In [401]:
cudosLong = cudosLong[-which(rowSums(is.na(cudosLong)) >= 18),]

In [417]:
cudosLong %>%
    group_by(ID)%>%
    summarise(maxday = max(day)) %>%
    summarize(mean = mean(maxday), sd = sd(maxday), min(maxday), max(maxday))

`summarise()` ungrouping output (override with `.groups` argument)



mean,sd,min(maxday),max(maxday)
<dbl>,<dbl>,<dbl>,<dbl>
7.212211,5.714815,0,57


In [424]:
cudosLong$DEPMEAN = rowMeans(cudosLong[,grep('dep', colnames(cudosLong))], na.rm = T)

In [429]:
gmm1 = hlme(DEPMEAN~day, subject = 'ID', random = ~1, ng = 1, data = cudosLong)


Be patient, hlme is running ... 
The program took 9.6 seconds 


In [431]:
gmm2 = gridsearch(rep = 10, maxiter = 10, minit = gmm1, 
                  hlme(DEPMEAN~day, subject = "ID", 
                       random=~1, ng = 2, data = cudosLong, mixture = ~ day, nwg=T, verbose = F))


In [440]:
gmm1$BIC
gmm2$BIC
gmm3$BIC

[1] 62183.97

[1] 58387.39

In [441]:
gmm3 = gridsearch(rep = 10, maxiter = 10, minit = gmm1, 
                  hlme(DEPMEAN~day, subject = "ID", 
                       random=~1, ng = 3, data = cudosLong, mixture = ~day, nwg=T, verbose = F))


In [444]:
gmm4 = gridsearch(rep = 10, maxiter = 10, minit = gmm1, 
                  hlme(DEPMEAN~day, subject = "ID", 
                       random=~1, ng = 4, data = cudosLong, mixture = ~day, nwg=T, verbose = F))


Heterogenous linear mixed model 
     fitted by maximum likelihood method 
 
hlme(fixed = DEPMEAN ~ day, mixture = ~day, random = ~1, subject = "ID", 
    ng = 3, nwg = T, data = cudosLong, verbose = F)
 
Statistical Model: 
     Dataset: cudosLong 
     Number of subjects: 4750 
     Number of observations: 34017 
     Number of latent classes: 3 
     Number of parameters: 12  
 
Iteration process: 
     Convergence criteria satisfied 
     Number of iterations:  4 
     Convergence criteria: parameters= 8.6e-11 
                         : likelihood= 6.2e-09 
                         : second derivatives= 1.8e-14 
 
Goodness-of-fit statistics: 
     maximum log-likelihood: -28616.84  
     AIC: 57257.68  
     BIC: 57335.27  
 
 